In [4]:
import ccxt
import time

# API Keys (Replace with actual keys)
BYBIT_API_KEY = 'I6m01uNAWiensHbIBA'
BYBIT_API_SECRET = 'vkaviS9xw903NJPncMtHgGfyiWB4mnhSJTyK'
BITGET_API_KEY = 'bg_b0a9d721607623216a87b066a3df34c3'
BITGET_API_SECRET = '6c4a5016562624b6d7f3b1c59563d4a1a573f630e0e49422fbf509399e56918b'

# Connect to exchanges
bybit = ccxt.bybit({
    'apiKey': BYBIT_API_KEY,
    'secret': BYBIT_API_SECRET,
    'options': {'adjustForTimeDifference': True}
})

bitget = ccxt.bitget({
    'apiKey': BITGET_API_KEY,
    'secret': BITGET_API_SECRET,
    'options': {'adjustForTimeDifference': True}
})


In [5]:
# Arbitrage settings
SPREAD_THRESHOLD = 0.5 / 100  # 0.5% profit target
MIN_VOLUME = 50000  # Minimum 24h volume required (in USDT)
CHECK_INTERVAL = 5  # Check every 5 seconds
TRADING_FEE = 0.001  # 0.1% trading fee per trade

def get_tradable_tokens(exchange, exchange_name):
    """Fetch tradable spot and futures tokens"""
    try:
        markets = exchange.load_markets()
        tokens = {"spot": [], "futures": []}

        for symbol in markets:
            if "/USDT" in symbol and ":" not in symbol:
                # Spot pair
                ticker = exchange.fetch_ticker(symbol)
                volume = ticker.get('quoteVolume', 0) or ticker.get('baseVolume', 0)
                if volume and volume >= MIN_VOLUME:
                    tokens["spot"].append(symbol)
            elif ":USDT" in symbol:
                # Futures pair
                ticker = exchange.fetch_ticker(symbol)
                volume = ticker.get('quoteVolume', 0) or ticker.get('baseVolume', 0)
                if volume and volume >= MIN_VOLUME:
                    tokens["futures"].append(symbol)

        print(f"✅ Found {len(tokens['spot'])} spot tokens and {len(tokens['futures'])} futures tokens on {exchange_name}")
        return tokens
    except Exception as e:
        print(f"❌ Error fetching tokens from {exchange_name}: {e}")
        return {"spot": [], "futures": []}

def get_price(exchange, symbol):
    """Get latest price for symbol"""
    try:
        if symbol in exchange.load_markets():
            ticker = exchange.fetch_ticker(symbol)
            return float(ticker.get('last', ticker.get('lastPrice', 0)))
        return None
    except Exception as e:
        print(f"❌ Error fetching price for {symbol}: {e}")
        return None

def check_arbitrage(symbol_spot, symbol_futures=None):
    """Check spot-spot and spot-futures arbitrage"""
    # Prices
    bybit_spot = get_price(bybit, symbol_spot)
    bitget_spot = get_price(bitget, symbol_spot)

    # Futures
    bybit_futures = get_price(bybit, symbol_futures) if symbol_futures else None
    bitget_futures = get_price(bitget, symbol_futures) if symbol_futures else None

    trade_opps = []

    if bybit_spot and bitget_spot:
        trade_opps.append(('Bybit Spot', 'Bitget Spot', bybit_spot, bitget_spot))
        trade_opps.append(('Bitget Spot', 'Bybit Spot', bitget_spot, bybit_spot))

    if bybit_spot and bitget_futures:
        trade_opps.append(('Bybit Spot', 'Bitget Futures', bybit_spot, bitget_futures))

    if bitget_spot and bybit_futures:
        trade_opps.append(('Bitget Spot', 'Bybit Futures', bitget_spot, bybit_futures))

    for buy_exchange, sell_exchange, buy_price, sell_price in trade_opps:
        spread = (sell_price - buy_price) / buy_price
        net_profit = spread - (TRADING_FEE * 2)

        if net_profit >= SPREAD_THRESHOLD:
            print(f"🔍 Arbitrage Detected: Buy on {buy_exchange}, Sell on {sell_exchange} for {symbol_spot}")
            print(f"    ✅ Buy Price: {buy_price:.5f} USDT")
            print(f"    ✅ Sell Price: {sell_price:.5f} USDT")
            print(f"    ✅ Spread: {spread:.4%}")
            print(f"    ✅ Net Profit After Fees: {net_profit:.4%}")
            print("-" * 50)

while True:
    try:
        # Load tradable tokens
        bybit_tokens = get_tradable_tokens(bybit, "Bybit")
        bitget_tokens = get_tradable_tokens(bitget, "Bitget")

        # Find common spot pairs
        common_spot = list(set(bybit_tokens["spot"]) & set(bitget_tokens["spot"]))
        common_futures = list(set(bybit_tokens["futures"]) | set(bitget_tokens["futures"]))

        if not common_spot:
            print("❌ No common spot tokens found. Retrying...")
            time.sleep(10)
            continue

        print("🔄 Scanning arbitrage opportunities...")

        for symbol in common_spot:
            fut_symbol = symbol.replace("/USDT", ":USDT")
            if fut_symbol in common_futures:
                check_arbitrage(symbol_spot=symbol, symbol_futures=fut_symbol)
            else:
                check_arbitrage(symbol_spot=symbol)

        print("✅ Cycle complete. Waiting...")
        time.sleep(CHECK_INTERVAL)

    except Exception as e:
        print(f"❌ Error in main loop: {e}")
        time.sleep(10)

❌ Error fetching tokens from Bybit: bybit GET https://api.bybit.com/v5/market/tickers?symbol=BTC-25APR25-74000-P-USDT&category=option
❌ Error fetching tokens from Bitget: bitget GET https://api.bitget.com/api/v2/spot/public/coins
❌ No common spot tokens found. Retrying...
❌ Error fetching tokens from Bybit: bybit GET https://api.bybit.com/v5/market/tickers?symbol=BTCUSDT&category=spot
❌ Error fetching tokens from Bitget: bitget GET https://api.bitget.com/api/v2/spot/public/coins
❌ No common spot tokens found. Retrying...
✅ Found 430 spot tokens and 589 futures tokens on Bybit
✅ Found 551 spot tokens and 427 futures tokens on Bitget
🔄 Scanning arbitrage opportunities...
🔍 Arbitrage Detected: Buy on Bybit Spot, Sell on Bitget Spot for ZENT/USDT
    ✅ Buy Price: 0.01268 USDT
    ✅ Sell Price: 0.01281 USDT
    ✅ Spread: 1.0252%
    ✅ Net Profit After Fees: 0.8252%
--------------------------------------------------
🔍 Arbitrage Detected: Buy on Bybit Spot, Sell on Bitget Spot for NESS/USDT
 

KeyboardInterrupt: 